# Hệ thống gợi ý phim sử dụng TF-IDF và Cosine Similarity

### 1. KẾT NỐI CƠ SỞ DỮ LIỆU MYSQL

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import mysql.connector

# Database connection function
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="csdl_phim"
    )

# Load data from MySQL
def load_movies_data():
    conn = get_db_connection()
    query = """
    SELECT 
        p.id, 
        p.ten_phim AS title, 
        MAX(p.mo_ta) AS description, 
        GROUP_CONCAT(DISTINCT tl.ten_the_loai SEPARATOR ' ') AS genres
    FROM 
        phims p
    LEFT JOIN 
        chi_tiet_the_loais ctl ON p.id = ctl.id_phim
    LEFT JOIN 
        the_loais tl ON ctl.id_the_loai = tl.id
    GROUP BY 
        p.id, p.ten_phim
    """
    movies = pd.read_sql(query, conn)
    conn.close()
    return movies
    


### 2. Tạo ma trận đặc trưng cho phim

In [2]:

def create_feature_matrix(movies_df):
    # Kết hợp cột thể loại và mô tả phim thành một văn bản duy nhất
    movies_df['combined_features'] = movies_df['genres'].fillna('') + ' ' + movies_df['description'].fillna('')
    
    # Khởi tạo TF-IDF Vectorizer
    tfidf = TfidfVectorizer(
        stop_words='english',  # Loại bỏ các từ không quan trọng trong tiếng Anh
        ngram_range=(1, 2),    # Sử dụng cả unigram và bigram
        min_df=2               # Bỏ qua các từ xuất hiện quá ít
    )
    
    # Biến đổi văn bản thành ma trận vector TF-IDF
    feature_matrix = tfidf.fit_transform(movies_df['combined_features'])
    return feature_matrix


### 3. Gợi ý phim dựa trên một bộ phim

In [6]:
def get_recommendations(movie_id, feature_matrix, movies_df, n=5):
    try:
        # Tính ma trận độ tương đồng Cosine giữa các phim
        similarity = cosine_similarity(feature_matrix)
        
        # Tìm vị trí (index) của bộ phim trong DataFrame
        movie_idx = movies_df[movies_df['id'] == movie_id].index[0]
        
        # Lấy danh sách các điểm similarity với phim đầu vào
        sim_scores = list(enumerate(similarity[movie_idx]))
        
        # Sắp xếp theo điểm similarity giảm dần (bỏ qua phim chính nó)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
        
        # Lấy danh sách các index phim được gợi ý
        movie_indices = [i[0] for i in sim_scores] 
        
        # Trả về danh sách phim được gợi ý
        recommendations = movies_df.iloc[movie_indices][['id', 'title', 'genres']].to_dict('records')
        
        # Thêm điểm similarity vào kết quả
        for idx, rec in enumerate(recommendations):
            rec['similarity_score'] = float(sim_scores[idx][1])
            
        return recommendations
        
    except Exception as e:
        print(f"Error in get_recommendations: {str(e)}")
        return []



### 4. Lấy danh sách phim người dùng đã xem gần đây

In [5]:
def get_user_recently_watched(user_id, limit=5):
    """Lấy danh sách phim user đã xem gần đây"""
    conn = get_db_connection()
    query = """
        SELECT 
            p.id,
            p.ten_phim AS title,
            MAX(p.mo_ta) AS description,
            GROUP_CONCAT(DISTINCT tl.ten_the_loai SEPARATOR ', ') AS genres,
            lx.ngay_xem
        FROM 
            luot_xems lx
            JOIN phims p ON lx.id_phim = p.id
            LEFT JOIN chi_tiet_the_loais ctl ON p.id = ctl.id_phim
            LEFT JOIN the_loais tl ON ctl.id_the_loai = tl.id
        WHERE 
            lx.id_khach_hang = %s
            AND lx.ngay_xem >= DATE_SUB(NOW(), INTERVAL 30 DAY)
        GROUP BY 
            p.id, p.ten_phim, lx.ngay_xem
        ORDER BY 
            lx.ngay_xem DESC
        LIMIT %s
    """
    
    recently_watched_df = pd.read_sql(query, conn, params=(user_id, limit))
    conn.close()
    return recently_watched_df



### 5. Gợi ý phim dựa trên nhiều phim được xem gần đây

In [7]:

def get_recommendations_from_multiple_movies(movie_ids, feature_matrix, movies_df, n=5):
    """Lấy recommendations dựa trên nhiều phim"""
    try:
        # Tính similarity matrix
        similarity = cosine_similarity(feature_matrix)
        
        # Tạo vector lưu tổng điểm similarity của tất cả phim đầu vào
        combined_scores = np.zeros(len(movies_df))
        
        for movie_id in movie_ids:
            movie_idx = movies_df[movies_df['id'] == movie_id].index[0]
            combined_scores += similarity[movie_idx]  # Cộng điểm similarity
        
        # Tính điểm similarity trung bình
        combined_scores = combined_scores / len(movie_ids)
        
        # Tạo danh sách index và điểm similarity
        sim_scores = list(enumerate(combined_scores))
        
        # Loại bỏ các phim đã xem
        sim_scores = [s for s in sim_scores if movies_df.iloc[s[0]]['id'] not in movie_ids]
        
        # Sắp xếp theo điểm similarity giảm dần
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:n]
        
        # Lấy danh sách phim được gợi ý
        movie_indices = [i[0] for i in sim_scores]
        recommendations = movies_df.iloc[movie_indices][['id', 'title', 'genres']].to_dict('records')
        
        # Thêm điểm similarity vào kết quả
        for idx, rec in enumerate(recommendations):
            rec['similarity_score'] = float(sim_scores[idx][1])
            
        return recommendations
        
    except Exception as e:
        print(f"Error in get_recommendations_from_multiple_movies: {str(e)}")
        return []


# GỢI Ý PHIM TƯƠNG TỰ


In [8]:
movie_id = 33
movies_df = load_movies_data()
feature_matrix = create_feature_matrix(movies_df)
recommendations = get_recommendations(
        movie_id=movie_id,
        feature_matrix=feature_matrix,
        movies_df=movies_df,
        n=5
    )
# Chuyển recommendations thành DataFrame rồi xem
recommendations_df = pd.DataFrame(recommendations)
movies_df.head(5)

C:\Users\VANVIET\AppData\Local\Temp\ipykernel_7312\2413871904.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movies = pd.read_sql(query, conn)


,id,title,description,genres,combined_features
0,1,"Suỵt, Quốc Vương Đang Ngủ Đông",Tác giả truyện tranh Vệ Chi bất ngờ vướng vào ...,Chính Kịch,Chính Kịch Tác giả truyện tranh Vệ Chi bất ngờ...
1,2,Xứ sở robot,Một thiếu nữ mồ côi cùng một robot bí ẩn lên đ...,Chính Kịch Khoa Học Phiêu Lưu Viễn Tưởng,Chính Kịch Khoa Học Phiêu Lưu Viễn Tưởng Một t...
2,3,Địch Nhân Kiệt: Thông Thiên Huyền Án,"Vào thời nhà Đường, Công chúa Thái Bình có một...",Bí Ẩn Chính Kịch Hành Động,"Bí Ẩn Chính Kịch Hành Động Vào thời nhà Đường,..."
3,4,Xuân Nương,Cuộc đại lễ đúc liên quan đến vận mệnh quốc gi...,Bí Ẩn Hài Hước,Bí Ẩn Hài Hước Cuộc đại lễ đúc liên quan đến v...
4,5,Vị Đại Hiệp Này Có Chút Nhát,"Vạn Tiểu Hoa là một hiệp khách giang hồ, nhưng...",Hài Hước Hành Động,Hài Hước Hành Động Vạn Tiểu Hoa là một hiệp kh...


# GỢI Ý PHIM DỰA TRÊN PHIM ĐÃ XEM GẦN ĐÂY


In [9]:
recently_watched = get_user_recently_watched(23)
recent_movie_ids = recently_watched['id'].tolist()
recommendations_watched = get_recommendations_from_multiple_movies(
            movie_ids=recent_movie_ids,
            feature_matrix=feature_matrix,
            movies_df=movies_df,
            n=5
        )
recommendations_watched = pd.DataFrame(recommendations_watched)
recommendations_watched.head(5)

C:\Users\VANVIET\AppData\Local\Temp\ipykernel_7312\3783155484.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recently_watched_df = pd.read_sql(query, conn, params=(user_id, limit))


,id,title,genres,similarity_score
0,94,Trêu Nhầm Sắc Son,Chính Kịch,0.175567
1,24,Chu Công Phục Yêu,Hành Động,0.161050
2,41,Khi Cuộc Đời Cho Bạn Quả Quýt,Chính Kịch,0.156077
3,22,Ti Thừa Đại Nhân Thân Yêu,Chính Kịch,0.147979
4,68,Kiến Khanh Khanh,Chính Kịch,0.136598
